In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
import requests
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
filename = "spacex_launch_dash.csv"
response = requests.get(url)
if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"Downloaded: {filename}")

Downloaded: spacex_launch_dash.csv


In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(filename)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
site_options = [{'label': 'All Sites', 'value':'All Sites'}] + [
    {'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()
        ]
site_options

[{'label': 'All Sites', 'value': 'All Sites'},
 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}]

In [5]:
max_payload

9600.0

In [6]:
site_count = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40']
site_count_group = site_count.groupby(["Launch Site","class"]).size().reset_index()
list(site_count_group[0].values)
site_count_group.head()

,Launch Site,class,0
0,CCAFS LC-40,0,19
1,CCAFS LC-40,1,7


In [7]:
# Create a dash application
app = dash.Dash(__name__)

In [8]:
# Create an app layout
app.layout = html.Div(
        children=[html.H1('SpaceX Launch Records Dashboard',
                style={'textAlign': 'center', 'color': '#503D36',
                       'font-size': 40}),
        # TASK 1: Add a dropdown list to enable Launch Site selection
        # The default select value is for ALL sites
        # dcc.Dropdown(id='site-dropdown',...)
           html.Div(       
                  dcc.Dropdown(
            id='site-dropdown',
            options=site_options,
            value='All Sites',
            placeholder='Select a launch site',
                      searchable=True
        ) ),
        html.Br(),

        # TASK 2: Add a pie chart to show the total successful launches count for all sites
        # If a specific launch site was selected, show the Success vs. Failed counts for the site
        html.Div(dcc.Graph(id='success-pie-chart')),
        html.Br(),

        html.P("Payload range (Kg):"),
        # TASK 3: Add a slider to select payload range
        html.Div([
        dcc.RangeSlider(id='payload-slider',
        min=0, max=10000, step=1000
        ,marks={i: str(i) for i in range(int(min_payload), int(max_payload) + 1, 1000)}
,
        value=[int(min_payload), int(max_payload)]),

    html.Div(id='slider-output')  # This will show the selected range
]),

        # TASK 4: Add a scatter chart to show the correlation between payload and launch success
        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
        ])


In [9]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure')
    , Input(component_id='site-dropdown', component_property='value'))


def update_pie_chart(Select_Site ):
    if Select_Site == 'All Sites':

        success_df = spacex_df[spacex_df['class'] == 1]
        success_df_group = spacex_df.groupby(["Launch Site"])["class"].sum().reset_index()
        
        fig=px.pie(success_df_group,
                    values='class',
                 names='Launch Site',
                 title="Total Successful Launches Count for All Sites"
                )
        return fig
    else:
        site_count = spacex_df[spacex_df['Launch Site'] == Select_Site]
        site_count_group = site_count.groupby(["Launch Site","class"]).size().reset_index()
        fig=px.pie(site_count_group,
                    values=list(site_count_group[0].values),
                 names=["Failure","Success"],
                 title="Success vs. Failed counts for site: "+Select_Site
                )
        return fig   

In [10]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),[Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])


def update_scatter_chart(Select_Site, Payload_Range ):
    if Select_Site == 'All Sites':

        df = spacex_df[(spacex_df['Payload Mass (kg)'] >=  Payload_Range[0]) &  (spacex_df['Payload Mass (kg)'] <=  Payload_Range[1])]
        fig = px.scatter(df,x = 'Payload Mass (kg)', y = 'class', color = "Booster Version Category",title = "Correlation between Payload and Success for all sites")
        return fig 
    else:
        df = spacex_df[(spacex_df['Payload Mass (kg)'] >=  Payload_Range[0]) &  (spacex_df['Payload Mass (kg)'] <=  Payload_Range[1]) & (spacex_df['Launch Site'] == Select_Site)]
        fig = px.scatter(df,x = 'Payload Mass (kg)', y = 'class', color = "Booster Version Category",title = "Correlation between Payload and Success for site: "+Select_Site)
        return fig   

In [11]:
# Run the app
import webbrowser
import threading
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")
if __name__ == '__main__':
    threading.Timer(1, open_browser).start()
    app.run()